In [7]:
#Particle class from lecture with z-component added
class Particle:
    def __init__(self, x, y, z, vx, vy, vz, mass):
        self.x = x
        self.y = y
        self.z = z
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.mass = mass
        
    def implicit_update(self, Fx, Fy, Fz, dt):
        self.vx = self.vx + Fx / self.mass * dt
        self.vy = self.vy + Fy / self.mass * dt
        self.vz = self.vz + Fz / self.mass * dt
        self.x = self.x + self.vx * dt
        self.y = self.y + self.vy * dt
        self.z = self.z + self.vz * dt
    
    def euler_update(self, Fx, Fy, Fz, dt):
        self.x = self.x + self.vx * dt
        self.y = self.y + self.vy * dt
        self.z = self.z + self.vz * dt
        self.vx = self.vx + Fx / self.mass * dt
        self.vy = self.vy + Fy / self.mass * dt
        self.vz = self.vz + Fz / self.mass * dt
    
    def leapfrog_update_pos(self, dt):
        self.x = self.x + self.vx * .5 * dt
        self.y = self.y + self.vy * .5 * dt
        self.z = self.z + self.vz * .5 * dt
        
    def leapfrog_update_velocity(self, Fx, Fy, Fz, dt):
        self.vx = self.vx + Fx / self.mass * dt
        self.vy = self.vy + Fy / self.mass * dt
        self.vz = self.vz + Fz / self.mass * dt
        
    def pairwise_force(self, particle):
        G = 6.674 * (10**-11)
        r2 = (self.x - particle.x)**2.0 + \
             (self.y - particle.y)**2.0 + \
             (self.z - particle.z)**2.0
        F_mag = -(G * particle.mass * self.mass * r2**.5)/(r2+100)**1.5
        F_x = (self.x - particle.x)/r2**0.5 * F_mag
        F_y = (self.y - particle.y)/r2**0.5 * F_mag
        F_z = (self.z - particle.z)/r2**0.5 * F_mag
        return (F_x, F_y, F_z)
    def distance(self, particle):
        return((self.x-particle.x)**2 + (self.y-particle.y)**2 + (self.z-particle.z)**2)**.5
    def print_particle(self):
        print("Position:", self.x, ",", self.y, ",", self.z)
        print("Velocity:", self.vx, ",", self.vy, ",", self.vz)

In [9]:
class Engine:
    def __init__(self, background, particles, strictly_background):
        self.background = background
        self.particles = particles
        self.strictly_background = strictly_background;
        self.t = 0.0
        self.iteration = 0
    def print_positions(self):
        for particle in self.particles:
            particle.print_particle()
            
    def plot_positions(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax = fig.gca(projection='3d')        
        ax.set_xlim(-10, 10)
        ax.set_ylim(-10, 10)
        ax.set_zlim(-10, 10)
        xs = []
        ys = []
        zs = []
        for particle in self.particles:
            xs.append(particle.x)
            ys.append(particle.y)
            zs.append(particle.z)
        ax.scatter(xs,ys,zs)
        name = 'fig'+str(self.iteration)+'.png'
        fig.savefig(name)
        plt.close(fig)
        self.iteration += 1
        
    def euler(self, timestep, integrator):
        self.t += timestep
        for particle in self.particles:
            if integrator == 2:
                particle.leapfrog_update_pos(timestep)
                
            F_x_total = background[0]
            F_y_total = background[1]
            F_z_total = background[2]
            if not self.strictly_background:
                for other in self.particles:
                    if other is particle:
                            continue
                    F_x, F_y, F_z = particle.pairwise_force(other)
                    F_x_total += F_x
                    F_y_total += F_y
                    F_z_total += F_z
            
            if integrator == 0: 
                particle.euler_update(F_x_total, F_y_total, F_z_total, timestep)
            elif integrator == 1:
                particle.implicit_update(F_x_total, F_y_total, F_z_total, timestep)
            elif integrator == 2:
                particle.leapfrog_update_velocity(F_x_total, F_y_total, F_z_total, timestep)
                particle.leapfrog_update_pos(timestep)
                
            

In [42]:
##TESTING TIME FOR DIFFERENT CASES##
##MUST RUN PARTICLE AND ENGINE CELLS##
import numpy as np
import time
#Number of particles and#
#Random number seed     #
#########################
number = 20
seed = 0x4d3d3d3
np.random.seed(seed)


#########################
#BACKGROUND FIELD AND   #
#TOGGLE ONLY BACKGROUND #
#########################z
background = [0,0,0]
strictly_background = False

##########################
#INTEGRATOR SELECTION    #
#0 = direct euler        #
#1 = implicit euler      #
#2 = leapfrog            #
##########################
integrator = 0

def test(background, strictly_background, integrator, n):
    if integrator == 0:
        integratorString = "Direct Euler"
    elif integrator == 1:
        integratorString = "Implicit Euler"
    elif integrator == 2:
        integratorString = "Leapfrog"


    # Create random data
    positions = np.random.uniform(-10, 10, size=(n, 3))
    velocities = np.random.uniform(0, 0, size=(n, 3))
    masses = np.random.uniform(10000000, 10000000000, size=(n, 1))

    particles = []
    for i in range (len(positions)):
        particle = Particle(positions[i][0], positions[i][1], positions[i][2], 
                            velocities[i][0], velocities[i][1], velocities[i][2],
                            masses[i])
        particles.append(particle)

    engine = Engine(background, particles, strictly_background)

    runtime = 5.0
    step = .5
    startTime = time.time()
    while (runtime > engine.t):
        engine.euler(step, integrator)
    endTime = time.time()
    print("Time for " + str(n) + " particles with " + integratorString + " integrator took: " + str(endTime-startTime) + " seconds!")

print("This will run the simulation for each integration method with a runtime of 5.0 and a stepsize of .5\n")
    
test(background, strictly_background, 0, 5)
test(background, strictly_background, 0, 10)
test(background, strictly_background, 0, 20)
test(background, strictly_background, 0, 40)
test(background, strictly_background, 0, 80)
test(background, strictly_background, 0, 160)

print("")

test(background, strictly_background, 1, 5)
test(background, strictly_background, 1, 10)
test(background, strictly_background, 1, 20)
test(background, strictly_background, 1, 40)
test(background, strictly_background, 1, 80)
test(background, strictly_background, 1, 160)

print("")

test(background, strictly_background, 2, 5)
test(background, strictly_background, 2, 10)
test(background, strictly_background, 2, 20)
test(background, strictly_background, 2, 40)
test(background, strictly_background, 2, 80)
test(background, strictly_background, 2, 160)

print("\n")
print("Our integrators were expected to run on O(N^2), and these data shows it!")
print("Despite doubling the number of particles each iteration, the ")
print("amount of time grew much faster (exponentially) for all three integration methods")
print("\n\n")


print("Lets compare the runtime of each integration method for a large N\n")
test(background, strictly_background, 0, 222)
test(background, strictly_background, 1, 222)
test(background, strictly_background, 2, 222)
print("\nEven for a large N the amount of time is pretty similar, which is\nto be expected as each integrator was O(N^2)\nand none of the integrators had any significant overhead")

This will run the simulation for each integration method with a runtime of 5.0 and a stepsize of .5

Time for 5 particles with Direct Euler integrator took: 0.004271745681762695 seconds!
Time for 10 particles with Direct Euler integrator took: 0.01797771453857422 seconds!
Time for 20 particles with Direct Euler integrator took: 0.0706186294555664 seconds!
Time for 40 particles with Direct Euler integrator took: 0.2928595542907715 seconds!
Time for 80 particles with Direct Euler integrator took: 1.3058922290802002 seconds!
Time for 160 particles with Direct Euler integrator took: 4.751773357391357 seconds!

Time for 5 particles with Implicit Euler integrator took: 0.004185676574707031 seconds!
Time for 10 particles with Implicit Euler integrator took: 0.018024921417236328 seconds!
Time for 20 particles with Implicit Euler integrator took: 0.07229280471801758 seconds!
Time for 40 particles with Implicit Euler integrator took: 0.31363821029663086 seconds!
Time for 80 particles with Implic